### Taking configuration parameter values from config.json

In [3]:
## Loading the configurations from config.json file.
import json
with open("config.json") as file:
    build_parameters = json.load(file)

In [4]:
import boto3
import sagemaker

region = boto3.Session().region_name
pipeline_output_bucket = build_parameters["output_bucket"] 
sagemaker_session = sagemaker.session.Session(default_bucket = pipeline_output_bucket)
# sagemaker_session = sagemaker.session.Session()
# role = sagemaker.get_execution_role()
role = "arn:aws:iam::852619674999:role/service-role/AmazonSageMaker-ExecutionRole-20220427T124311"

print(role)
print(sagemaker_session.default_bucket())

arn:aws:iam::852619674999:role/service-role/AmazonSageMaker-ExecutionRole-20220427T124311
mloutput-churn-telecomchurn-852619674999-ap-south-1


### Handling Input

In [5]:
input_bucket = build_parameters["input_bucket"]
batch_data_file_name = build_parameters["scoring_data_file_name"]
batch_data_uri = f"s3://{input_bucket}/{batch_data_file_name}"
# batch_data_uri = build_parameters["scoring_data_s3_location"]

from sagemaker.workflow.parameters import ParameterInteger, ParameterString

batch_data = ParameterString(name="BatchData", default_value=batch_data_uri)
feature_selection_file_name = build_parameters["feature_selection_file_name"]
# input_feature_selection_file_uri = build_parameters["feature_selection_file_s3_location"]
input_feature_selection_file_uri = f"s3://{input_bucket}/{feature_selection_file_name}"

# preprocessing_code_location_uri = f"s3://{pipeline_input_bucket}/codes/Training_Preprocessing.py"

# Basic feature selection file path
feature_selection_file = ParameterString(name = "FeatureSelectionFile", default_value = input_feature_selection_file_uri)

### Handling Output

In [6]:

# sagemaker_session.default_bucket = pipeline_output_bucket

# Creating the output bucket if it is not already present
s3 = boto3.client('s3')
buckets = [dictionary["Name"] for dictionary in s3.list_buckets()['Buckets']]
if pipeline_output_bucket not in buckets:
    location = {'LocationConstraint': region}
    response = s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration = location)


from time import gmtime, strftime
pipeline_start_time = strftime("%Y%m%d-%H-%M-%S", gmtime())

processing_output_path = f"s3://{pipeline_output_bucket}/Scoring_Pipeline_Output/{pipeline_start_time}/ProcessingOutput"
inference_output_path = f"s3://{pipeline_output_bucket}/Scoring_Pipeline_Output/{pipeline_start_time}/InferenceOutput"

### Building the Preprocessing Component

In [7]:
from sagemaker.sklearn.processing import SKLearnProcessor

framework_version = build_parameters["sklearn_processor_framework_version"]

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=build_parameters["scoring_preprocessing_instance_type"],
    instance_count=build_parameters["scoring_preprocessing_instance_count"],
    base_job_name="Churn-Inference-Preprocessing",
    role=role
)


from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TuningStep
    

step_process = ProcessingStep(
    name="Preprocessing",
    processor=sklearn_processor,
    inputs=[
      ProcessingInput(source=batch_data, destination="/opt/ml/processing/input"),  
      ProcessingInput(source=feature_selection_file, destination="/opt/ml/processing/input/feature_selection")
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination = processing_output_path),
        ProcessingOutput(output_name = "logs", source="/opt/ml/processing/logss", destination = processing_output_path)
    ],
    # code="SageMaker_Pipeline_Component_Codes/Scoring/Scoring_Preprocessing.py",
    code = f"s3://{build_parameters['input_bucket']}/codes/{build_parameters['scoring_preprocessing_code_location'].split('/')[-1]}",
    job_arguments = ["--batch_data_location", "/opt/ml/processing/input", "--target_column", "Churn",
                     "--feature_selection_file_location", "/opt/ml/processing/input/feature_selection",
                     "--preprocessed_batch_data_location", "/opt/ml/processing/train", "--log_location", "/opt/ml/processing/logss"]
)

### Get Model Step

In [8]:
#### Obtaining the model from Sagemaker model registry.
package_group = build_parameters["model_package_group_name"]

import boto3
client = boto3.client('sagemaker')
model_packages = client.list_model_packages(ModelPackageGroupName = package_group)


latest_package = model_packages["ModelPackageSummaryList"][0]
latest_package_arn = latest_package["ModelPackageArn"]

print(latest_package)
print(latest_package_arn)


latest_package_details = client.describe_model_package(ModelPackageName=latest_package_arn)

from sagemaker.model import Model
inference_model = Model(image_uri = latest_package_details['InferenceSpecification']['Containers'][0]['Image'], 
                        # source_dir = f"s3://{build_parameters['input_bucket']}/codes/scoring.tar.gz",
                        # entry_point = f"{build_parameters['scoring_code_location'].split('/')[-1]}",
                        # entry_point="../" + build_parameters["scoring_code_loaction"], 
                        model_data = latest_package_details['InferenceSpecification']['Containers'][0]["ModelDataUrl"], 
                        role = role,
                        sagemaker_session = sagemaker_session,
                        env = {'model_server_workers':'1',
                              'SAGEMAKER_MODEL_SERVER_WORKERS':'1'}
                       )



from sagemaker.inputs import CreateModelInput

inputs = CreateModelInput(
    # instance_type=build_parameters["scoring_instance_type"],
    instance_type="ml.t3.medium",
    # accelerator_type="ml.eia1.medium",
)


from sagemaker.workflow.steps import CreateModelStep

step_create_model = CreateModelStep(
    name="Get-Model",
    model=inference_model,
    inputs=inputs,
    depends_on  = [step_process]
)

{'ModelPackageGroupName': 'churn-packagegroup', 'ModelPackageVersion': 5, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:852619674999:model-package/churn-packagegroup/5', 'CreationTime': datetime.datetime(2022, 6, 30, 7, 40, 14, 674000, tzinfo=tzlocal()), 'ModelPackageStatus': 'Completed', 'ModelApprovalStatus': 'PendingManualApproval'}
arn:aws:sagemaker:us-east-1:852619674999:model-package/churn-packagegroup/5


### Making Inference Step

In [ ]:
step_transform = ProcessingStep(
    name="Inference",
    processor=sklearn_processor,
    inputs=[
      ProcessingInput(source=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri, destination="/opt/ml/processing/input/data"),  
      ProcessingInput(source=step_create_model.properties.PrimaryContainer.ModelDataUrl, destination="/opt/ml/processing/input/model_folder"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination = inference_output_path)
    ],
    # code="SageMaker_Pipeline_Component_Codes/Scoring/scoring.py",
    # code = f"s3://{build_parameters['input_bucket']}/codes/{build_parameters['scoring_preprocessing_code_location'].split('/')[-1]}",
    code = f"s3://{build_parameters['input_bucket']}/codes/scoring.py",
    job_arguments = ["--batch_data_location", "/opt/ml/processing/input/data", 
                     "--model_location", "/opt/ml/processing/input/model_folder",
                     "--predicted_data_location", "/opt/ml/processing/train", 
                     "--log_location", "/opt/ml/processing/logss"
                    ]
)

In [8]:

# from sagemaker.transformer import Transformer


# transformer = Transformer(
#     model_name=step_create_model.properties.ModelName,
#     instance_type=build_parameters["scoring_instance_type"],
#     instance_count=1,
#     output_path=inference_output_path,
#     base_transform_job_name = "Churn-Transformation",
#     # max_concurrent_transforms = 1,
#     # strategy = "SingleRecord"
# )

# from sagemaker.inputs import TransformInput
# from sagemaker.workflow.steps import TransformStep


# step_transform = TransformStep(
#     name="Inference",
#     transformer=transformer,
#     inputs=TransformInput(data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
#                           # data_type = "text/csv"
#                          ),
#     depends_on  = [step_process]
# )

### Building the Pipeline

In [9]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"Churn-Scoring"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        batch_data,
        feature_selection_file
    ],
    steps=[step_process, 
           step_create_model, 
           step_transform
          ]
)

### Uploading the Pipeline

In [10]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:852619674999:pipeline/churn-scoring',
 'ResponseMetadata': {'RequestId': '58538039-fd4c-40e5-b8bb-c83eb66a1562',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '58538039-fd4c-40e5-b8bb-c83eb66a1562',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '81',
   'date': 'Fri, 01 Jul 2022 12:36:42 GMT'},
  'RetryAttempts': 0}}